In [4]:
##%overwritefile
##%file:src/c_start_gdb.py
##%noruncode
    def _start_gdb(self):
        # Signal handlers are inherited by forked processes, and we can't easily
        # reset it from the subprocess. Since kernelapp ignores SIGINT except in
        # message handlers, we need to temporarily reset the SIGINT handler here
        # so that bash and its children are interruptible.
        sig = signal.signal(signal.SIGINT, signal.SIG_DFL)
        try:
            if hasattr(self, 'gdbwrapper'):
                if not self.gdbwrapper.child.terminated:
                    return
        finally:
            pass
        try:
            # self._write_to_stdout("------exec gdb-----\n")
            child = pexpect.spawn('gdb', ['-q'], echo=False,encoding='utf-8')
            self.gdbwrapper = IREPLWrapper(
                                    self._write_to_stdout,
                                    self._write_to_stderr,
                                    self._read_from_stdin,
                                    child, 
                                    replsetip=u'(gdb)',
                                    orig_prompt=u'(gdb)',
                                    prompt_change=None,
                                    extra_init_cmd='set pagination off',
                                    line_output_callback=self.process_output)
        except Exception as e:
            self._logln(" IREPLWrapper error! \n"+str(e))
            exitcode = 1
        finally:
            signal.signal(signal.SIGINT, sig)


[MyPython] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/C/src/c_start_gdb.py created successfully


In [5]:
##%overwritefile
##%file:src/c_do_replexecutegdb.py
##%noruncode
    def do_replexecutegdb(self, code, silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        self.silent = silent
        if not code.strip():
            return {'status': 'ok', 'execution_count': self.execution_count,
                    'payload': [], 'user_expressions': {}}

        interrupted = False
        try:
            # Note: timeout=None tells IREPLWrapper to do incremental
            # output.  Also note that the return value from
            # run_command is not needed, because the output was
            # already sent by IREPLWrapper.
            self.gdbwrapper.run_command(code.rstrip(), timeout=None)
        except KeyboardInterrupt:
            self.gdbwrapper.child.sendintr()
            interrupted = True
            self.gdbwrapper._expect_prompt()
            output = self.gdbwrapper.child.before
            self.process_output(output)
        except EOF:
            output = self.gdbwrapper.child.before + 'Restarting GDB'
            self._start_gdb()
            self.process_output(output)

        if interrupted:
            return {'status': 'abort', 'execution_count': self.execution_count}

        try:
            if code.rstrip().startswith('shell'):
                exitcode = int(self.gdbwrapper.run_command('shell echo $?').rstrip())
            else:
                exitcode = int(self.gdbwrapper.run_command('echo $?').rstrip())
        except Exception:
            exitcode = 1

        if exitcode:
            error_content = {'execution_count': self.execution_count,
                             'ename': '', 'evalue': str(exitcode), 'traceback': []}

            self.send_response(self.iopub_socket, 'error', error_content)
            error_content['status'] = 'error'
            return error_content
        else:
            return {'status': 'ok', 'execution_count': self.execution_count,
                    'payload': [], 'user_expressions': {}}


[MyPython] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/C/src/c_do_replexecutegdb.py created successfully


In [6]:
##%overwritefile
##%file:src/c_replgdb_sendcmd.py
##%noruncode
    def replgdb_sendcmd(self,code,silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        self._start_gdb()
        return self.do_replexecutegdb( code.replace('//%rungdb', ''), silent, store_history,
                   user_expressions, False)


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/C/src/c_replgdb_sendcmd.py created successfully
